In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.interpolate as interp
import scipy.integrate as integrate
import os
import theoryLya as tLya
import Spectrograph as spectrograph
import Survey as survey
import effectiveStatistics as effStats

os.environ['PATH'] = '/global/common/sw/cray/sles15/x86_64/texlive/live/gcc/8.2.0/tiozj27/bin/x86_64-linux/:{}'.format(os.environ['PATH'])

from matplotlib import rc
import matplotlib.cm as cm
rc('font',**{'size':'20','family':'serif','serif':['CMU serif']})
rc('mathtext', **{'fontset':'cm'})
rc('text', usetex=True)
rc('legend',**{'fontsize':'13'})

import camb
from camb import model, initialpower

In [2]:
from astropy.table import Table

tt=Table.read('lya-snr-guadalupe.fits')

qso=tt[ (tt['Z']>2.4)&(tt['Z']<2.6) ]

In [ ]:
theory=tLya.theoryLya()
DESI1_survey = survey.Survey(nzr_file='./nzs/nzr_qso.dat', survey_A=16000)
#SNR Values from Path are overidden for guadalupe data below
path = "/global/homes/n/nishant/DESI2_LyaProjections/DESI-II-lyaforecast/build/lib/desi_SNR/"
DESI_instrument = spectrograph.Spectrograph(theory, path)
#survey area in sq degrees
DESI1 = effStats.effectiveStatistics(theory, DESI1_survey, DESI_instrument)

In [ ]:
nu_n_guad = np.zeros(shape=len(qso))

for i, item in enumerate(qso):
    nu_n_guad[i] = DESI1.v_n(4000, 22.25, item['Z'], k=0.15, SNR=item['SNR'] * DESI1.theoryLya.meanFlux(item['Z'])**2)

In [ ]:
times = range(5000,17000,1000)
nu_n_guad2 = np.zeros(shape=(len(qso), len(times)))

for j, time in enumerate(times):
    for i, item in enumerate(qso):
        nu_n_guad2[i,j] = DESI1.v_n(4000, 22.25, item['Z'], k=0.15, SNR=item['SNR'] * np.sqrt(time/4000) * DESI1.theoryLya.meanFlux(item['Z'])**2)
        

    


In [ ]:
diff = np.zeros(shape=(len(qso), len(times)))
temp = nu_n_guad
for i in range(12):
    diff[:,i] = nu_n_guad2[:,i] - temp
    temp = nu_n_guad2[:,i]
    

In [ ]:
for i in range(12):
    
    plt.hist(diff[:,i], bins=np.linspace(min(diff.flatten()),max(diff.flatten()), 50), alpha=0.5, label=str(i+1) + " extra 1000 seconds")

plt.xlabel(r'$\Delta \nu_{n}$')
plt.ylabel('count')
plt.legend()
plt.show()

In [ ]:
plt.hist(-diff ,stacked=True, bins=np.linspace(min(-diff.flatten()),max(-diff.flatten()), 50), label=(str(i+1) + " extra 1000 seconds"))
plt.xlabel(r'$-\Delta \nu_{n}$')
plt.ylabel('PDF')
plt.legend()
plt.show()

In [ ]:
plt.hist(-diff,stacked=True, cumulative=True, bins=np.linspace(min(-diff.flatten()),max(-diff.flatten()), 200), label=(str(i+1) + " extra 1000 seconds"))
plt.xlabel(r'$-\Delta \nu_{n}$')
plt.ylabel('CDF')
plt.show()

In [ ]:
n, bins, patches = plt.hist(nu_n_guad, bins=50, density=True)
plt.xlabel(r'$\nu_{n}$')
plt.ylabel('PDF')
plt.show()

In [ ]:
n, bins, patches = plt.hist(nu_n_guad, bins=np.linspace(0,1,51), density=True, cumulative=True)
plt.xlabel(r'$\nu_{n}$')
plt.ylabel('CDF')
plt.show()